# Generate release notes

This notebook automates creating a draft for the latest release notes using `generate_release_objects.py` and OpenAI API. All you need to do is have a valid OpenAI API key and the URLs of latest GitHub release tags you want to include.

Release information is extracted from the release tags, and then sorted by label. Then, we put the information through OpenAI for preliminary editing. 

After running the notebook, you'll see new generated release notes added to the `~/site/releases` folder for you to clean up further, along with a live preview of the site generated by the notebook to get you started.

## Prerequisites

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Refer our release notes guide for the full steps needed before running this notebook:</b></span>
<br></br>
<a href="https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4" style="color: #DE257E;"><b>Create Release Notes</b></a></div>

- [ ] You should be on a separate branch associated to the story for the release notes. 
- [ ] You need to have a valid `OPENAI_API_KEY` in your `.env` file. Your `.env` file should preferably live in your `documentation` repo root folder so you don't have to enter in the location of the file when the script runs.
- [ ] You need to have the GitHub release tags created and the URLs ready to be inputted.

## Setting up

### Import release generation script

This cell imports our custom `generate_release_objects.py` script for use, including any necessary dependencies.

In [1]:
import generate_release_objects

### Set up OpenAI API 

First, you'll be asked to indicate the location of the `.env` file where your OpenAI API key lives so it can return the secrey key. 

**Leave blank to go with the default path:** `../../.env`

In [2]:
generate_release_objects.env_location = generate_release_objects.get_env_location()
generate_release_objects.setup_openai_api()

### Create categories from labels

This cell creates the main sections of the release notes based on the GitHub labels.

**`label_hierarchy` shows the order in which updates will be shown.**

In [ ]:
label_hierarchy = ["highlight", "deprecation", "bug", "enhancement", "documentation"]

### Collect GitHub URLs 

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.

**Example release URL:** https://github.com/validmind/documentation/releases/tag/v2.4.4

In [ ]:
generate_release_objects.github_urls = generate_release_objects.collect_github_urls() 

### Set the release date 
Running this cell will prompt you to enter the desired release date. 

**The default is 3 business days from today if you leave the prompt empty.**

In [ ]:
generate_release_objects.release_datetime = generate_release_objects.get_release_date()
generate_release_objects.formatted_release_date = generate_release_objects.release_datetime.strftime("%Y-%b-%d").lower()
generate_release_objects.original_release_date = generate_release_objects.release_datetime.strftime("%B %-d, %Y")

## Extracting PR information 

### Create release folder

These lines will create a folder inside of `~/site/releases` for the release notes. 

The folder name is the release date, as per our convention.

In [ ]:
output_file = generate_release_objects.create_release_folder(generate_release_objects.formatted_release_date)
print(f"{output_file} created.")

### Create the release notes file
This block writes the title of the release notes into the final release notes file.

In [ ]:
print("Generating & editing release notes ...")
generate_release_objects.create_release_qmd(output_file, generate_release_objects.original_release_date)

### Set up release notes components
`release_components` will contain all the components of the release notes in the form of a dictionary. 

Later, we will merge everything together to create the release notes.

In [ ]:
updated_components = generate_release_objects.update_release_components(
    generate_release_objects.release_components, 
    generate_release_objects.categories
    )
print(updated_components)
print(f"Available release components: {generate_release_objects.release_components}")

### Set the repository and tag name 
This block checks every URL and assigns its repo name, such as `documentation` or `backend`, and its tag name.

In [ ]:
generate_release_objects.set_names(generate_release_objects.github_urls)

### Extract PRs from each URL 
This block gathers all the pull requests from each release URL and stores them within the URL's object data.

In [ ]:
generate_release_objects.extract_urls(generate_release_objects.github_urls)

### Load PR data 

Using the JSON data from the PRs, this block extracts and stores information into each PR's object data.

In [ ]:
generate_release_objects.populate_data(generate_release_objects.github_urls)

## Drafting the release notes 

### Edit the release notes body 

Using the prompt below, this block feeds the body of each PR to ChatGPT for editing, skipping PRs labeled as `internal`. 

If you find that the output is not quite right, edit the prompt and play around with it.

In [ ]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

generate_release_objects.edit_release_notes(generate_release_objects.github_urls, editing_instructions_body)

### Try automated GitHub PR summary

Using the new github-actions bot, we can fetch their auto-generated summary. This code block fetches the summary.

In [ ]:
summary_instructions = """ 
Please turn this PR Summary into a summary for release notes, according to the following guidelines:
- Use simple and neutral language in the active voice.
- Change from numbered list format to paragraph-style text.
- Address users directly in the second person with "you".
- Use present tense by avoiding the use of "will".
"""

generate_release_objects.process_github_prs(generate_release_objects.github_urls, summary_instructions)

### Edit each title
This block does the same as above for the titles of each PR. The output below will show:
- The original PR title
- The title after some algorithmic changes
- The title after ChatGPT edits it

If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.

In [ ]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in generate_release_objects.github_urls:
    for pr in url.prs:
        if pr.data_json: 
            print(f"Editing title for PR #{pr.pr_number} in {pr.repo_name}...\n")
            pr.title = pr.data_json['title']
            pr.clean_title(editing_instructions_title)
            print("\n")

### Set labels for each PR 
This block takes the label data from each PR and assigns it to the PR.

In [ ]:

for url in generate_release_objects.github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.labels = [label['name'] for label in pr.data_json['labels']]
            print(f"PR #{pr.pr_number} from {pr.repo_name}: {pr.labels}\n")

### Assign PR details to PR 
This block compiles all the data we found earlier for each PR into one place. 

In [ ]:

for url in generate_release_objects.github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.pr_details = {
            'pr_number': pr.pr_number,
            'title': pr.cleaned_title,
            'full_title': pr.data_json['title'],
            'url': pr.data_json['url'],
            'labels': ", ".join(pr.labels),
            'notes': pr.edited_text
            }
            print(f"PR #{pr.pr_number} from {pr.repo_name} added.\n")


### Combine all PR data into the same release notes components
Now, we can take all the details we compiled above and append them to our final release notes components. Since we want to show features in order of importance, we sort by the priority of the label.

In [ ]:

for url in generate_release_objects.github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Adding PR #{pr.pr_number} from {pr.repo_name}...\n")
            assigned = False 
            for priority_label in label_hierarchy:
                if priority_label in pr.labels:
                    release_components[priority_label].append(pr.pr_details)
                    assigned = True
                    break
            if not assigned:
                release_components.setdefault('other', []).append(pr.pr_details)

## Adding release notes to the docs

### Write release notes to file 
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

In [ ]:
# Write categorized PRs to the file
with open(output_file, "a") as file:
    generate_release_objects.write_prs_to_file(file, release_components, generate_release_objects.label_to_category)
    print(f"Release notes added to {file.name}.")


### Update sidebar 
This block will go into our `_quarto.yml` file and add the new release notes so it shows up on the sidebar of the docsite. 

In [ ]:

def update_quarto_yaml(release_date):
    """Updates the _quarto.yml file to include the release notes file so it can be accessed on the website.

    Params:
        release_date - release notes use the release date as the file name.
    
    Modifies:
        _quarto.yml file
    """
    yaml_filename = "../../site/_quarto.yml"
    temp_yaml_filename = "../../site/_quarto_temp.yml"

    # Copy the original YAML file to a temporary file
    shutil.copyfile(yaml_filename, temp_yaml_filename)

    with open(temp_yaml_filename, 'r') as file:
        lines = file.readlines()

    # Format the release date for insertion into the YAML file
    formatted_release_date = release_date.strftime("%Y-%b-%d").lower()

    with open(yaml_filename, 'w') as file:
        add_release_content = False
        insert_index = -1

        for i, line in enumerate(lines):
            file.write(line)
            if line.strip() == "# MAKE-RELEASE-NOTES-EMBED-MARKER":
                add_release_content = True
                insert_index = i

            if add_release_content and i == insert_index:
                file.write(f'        - releases/{formatted_release_date}/release-notes.qmd\n')
                add_release_content = False

    # Remove the temporary file
    os.remove(temp_yaml_filename)
    
    print(f"Added release notes to _quarto.yml, line {insert_index + 2}")

update_quarto_yaml(generate_release_objects.release_datetime)

### Update index 
This block will go into our `index.qmd` file and add the new release notes so it shows up in the "Latest Releases" section, as well as remove the oldest release so that the tiles are uniform. 

In [ ]:
def update_index_qmd(release_date):
    """Updates the index.qmd file to include the new releases in `Latest Releases` and removes the oldest release from the list.

    Params:
        release_date - release notes use the release date as the file name.
    
    Modifies:
        index.qmd file
    """
    index_filename = "../../site/index.qmd"
    temp_index_filename = "../../site/index_temp.qmd"

    # Copy the original QMD file to a temporary file
    shutil.copyfile(index_filename, temp_index_filename)

    with open(temp_index_filename, 'r') as file:
        lines = file.readlines()

    # Format the release date for insertion into the QMD file
    formatted_release_date = release_date.strftime("%Y-%b-%d").lower()

    with open(index_filename, 'w') as file:
        add_release_content = False
        insert_index = -1

        for i, line in enumerate(lines):
            file.write(line)
            if line.strip() == "# MAKE-RELEASE-NOTES-LATEST-MARKER":
                add_release_content = True
                insert_index = i

            if add_release_content and i == insert_index:
                file.write(f'      - /releases/{formatted_release_date}/release-notes.qmd\n')
                add_release_content = False

    # Remove the temporary file
    os.remove(temp_index_filename)
    
    print(f"Added new release notes to index.qmd, line {insert_index + 2}")

    with open(index_filename, 'r') as file:
        updated_lines = file.readlines()

    with open(index_filename, 'w') as file:
        for i, line in enumerate(updated_lines):
            # Identify the marker line
            if line.strip() == "# MAKE-RELEASE-NOTES-OLDEST-MARKER":
                # Check if the line above exists and starts with a list indicator "-"
                if i > 0 and updated_lines[i - 1].strip().startswith("-"):
                    # Write all lines up to the one before the line to remove
                    file.writelines(updated_lines[:i - 1])
                    # Write the marker and subsequent lines
                    file.writelines(updated_lines[i:])
                    break
        else:
            # If no marker is found, rewrite the file as is
            file.writelines(updated_lines)

    print("Removed the oldest release note entry from index.qmd.")

update_index_qmd(generate_release_objects.release_datetime)

### Show files to commit 

In [ ]:
# After completing all tasks, print git status to show output files
try:
    result = subprocess.run(["git", "status", "--short"], check=True, text=True, capture_output=True)
    lines = result.stdout.split('\n')
    print("Files to commit:")
    for line in lines:
        if line.startswith((' M', '??', 'A ')):
            print(line)
except subprocess.CalledProcessError as e:
    print("Failed to run git status:", e)

### Preview and edit changes
Run this cell to preview your changes, and make edits to the release notes file you just generated. See our [internal guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) on editing release notes.

In [ ]:
%%bash
cd ../../site
quarto preview

## Next steps

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>When you're done with the preview, please restart the kernel.</b></span>
<br></br>
Make sure that when you commit your changes to remote, do NOT include the changes in the <code>internal/release-scripts/</code> folder as that would commit changed notebook with the output cells.</div>

Now that you've generated, previewed, and edited the release notes, it's time to send a commit and start a PR! 

- [ ] Make sure you're on the branch associated to the story for the release notes. 
- [ ] Double check our [internal guide](https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4) to make sure you've completed all the steps!